In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma deployment to GKE using vLLM on GPU



## Overview

This notebook demonstrates downloading and deploying Gemma, open models from Google DeepMind. In this guide we specifically use L4 GPUs but this guide should also work for A100(40 GB), A100(80 GB), H100(80 GB) GPUs.


### Objective

Deploy and run inference for serving Gemma with vLLM on GPUs.

### GPUs

GPUs let you accelerate specific workloads running on your nodes such as machine learning and data processing. GKE provides a range of machine type options for node configuration, including machine types with NVIDIA H100, L4, and A100 GPUs.

Before you use GPUs in GKE, we recommend that you complete the following learning path:

Learn about [current GPU version availability](https://cloud.google.com/compute/docs/gpus)

Learn about [GPUs in GKE](https://cloud.google.com/kubernetes-engine/docs/concepts/gpus)




### Pre requisites
- Install Google Cloud CLI
https://cloud.google.com/sdk/docs/install-sdk  (note: pre-installed in Cloud Shell)
- Install kubectl (note: pre-installed in Cloud Shell)
- Create a .env file with the following values


```
PROJECT_ID
REGION 
HF_TOKEN
KSA_NAME 
PROJECT_NUMBER
CLUSTER_NAME 

# Assuming the model is saved at : gs://{MODEL_BUCKET}/{MODEL_NAME}/{MODEL_VERSION}
MODEL_BUCKET 
MODEL_NAME
MODEL_VERSION
IMAGE_NAME
```




## Create a GKE cluster and node pool

In [6]:
!pip install python-dotenv

**Restart the runtime session**

In [7]:
from dotenv import load_dotenv
import os


load_dotenv()  # This loads the variables from .env into the environment

PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")
# HF_TOKEN = os.getenv("HF_TOKEN")
PROJECT_NUMBER = os.getenv("PROJECT_NUMBER")
KSA_NAME = os.getenv("KSA_NAME")
CLUSTER_NAME = os.getenv("CLUSTER_NAME")

# Assuming the model is saved at : gs://{MODEL_BUCKET}/{MODEL_NAME}/{MODEL_VERSION}
MODEL_BUCKET = os.getenv("MODEL_BUCKET")
MODEL_NAME = os.getenv("MODEL_NAME")
MODEL_VERSION = os.getenv("MODEL_VERSION")
IMAGE_NAME = os.getenv("IMAGE_NAME")


In [9]:
MODEL_VERSION

'full_merged_model'

In [ ]:
# The HuggingFace token used to download models.

# assert HF_TOKEN, "Set Hugging Face access token in `HF_TOKEN`."

# Set up gcloud.
!gcloud config set project {PROJECT_ID}
!gcloud services enable container.googleapis.com



In [ ]:
import datetime
import os


# # Create a unique cluster name to avoid conflicts.
# now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# CLUSTER_NAME=f"gke-gemma-cluster-test-{now}"

print(f"Creating cluster: {CLUSTER_NAME}")

! gcloud container clusters create {CLUSTER_NAME} \
    --project={PROJECT_ID} \
    --region={REGION} \
    --subnetwork="default" \
    --workload-pool={PROJECT_ID}.svc.id.goog \
    --release-channel=rapid \
    --num-nodes=4 \
    --enable-shielded-nodes \
    --shielded-secure-boot\
    --shielded-integrity-monitoring \
    --addons=GcsFuseCsiDriver

! gcloud container node-pools create gpupool \
    --accelerator=type=nvidia-l4,count=2,gpu-driver-version=latest \
    --project={PROJECT_ID} \
    --location={REGION} \
    --node-locations={REGION}-a \
    --cluster={CLUSTER_NAME} \
    --machine-type=g2-standard-24 \
    --num-nodes=1 \
    --shielded-secure-boot \
    --shielded-integrity-monitoring

! gcloud container clusters get-credentials {CLUSTER_NAME} --location {REGION}


In [10]:
! gcloud container clusters get-credentials {CLUSTER_NAME} --location {REGION}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for gke-gemma-cluster-test-20250806054246.


### (Optional, only needed if load model from HF) Create a Kubernetes secret for Hugging Face credentials

In [ ]:
# Create Kubernetes secret for Hugging Face credentials
# ! kubectl create secret generic hf-secret \
#     --from-literal=hf_api_token={HF_TOKEN} \
#     --dry-run=client -o yaml > hf-secret.yaml
# 
# ! kubectl apply -f hf-secret.yaml

In [ ]:
!kubectl create serviceaccount {KSA_NAME} 
    

In [ ]:
! gcloud projects add-iam-policy-binding projects/{PROJECT_ID} \
    --role=roles/container.clusterViewer \
    --member=principal://iam.googleapis.com/projects/{PROJECT_NUMBER}/locations/global/workloadIdentityPools/{PROJECT_ID}.svc.id.goog/subject/ns/default/sa/{KSA_NAME} \
    --condition=None

### (Optional)Copy Gemma3 model data to your bucket

In [ ]:
# Source model: gs://<source_bucket>/<model_name>/<model_version> 
## Assuming the model will be saved at : gs://{MODEL_BUCKET}/{MODEL_NAME}/{MODEL_VERSION}

#! gcloud storage cp --recursive gs://the-fine-tuners/gemma3-1b-vertex gs://{MODEL_BUCKET}

In [ ]:
! gcloud storage buckets add-iam-policy-binding gs://{MODEL_BUCKET} \
    --role=roles/storage.objectViewer \
    --member=principal://iam.googleapis.com/projects/{PROJECT_NUMBER}/locations/global/workloadIdentityPools/{PROJECT_ID}.svc.id.goog/subject/ns/default/sa/{KSA_NAME} \
    --condition=None

In [ ]:

V_MODEL_BUCKET = MODEL_BUCKET
V_MODEL_NAME =MODEL_NAME
V_MODEL_VERSION=MODEL_VERSION
V_IMAGE_NAME=IMAGE_NAME
V_KSA=KSA_NAME

In [ ]:
# @title Deploy Gemma3

# @markdown This section deploys Gemma.

# @markdown Select one of the following model version and size options:

# The size of the model to launch


K8S_YAML_GCS=f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: vllm-gemma3-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: gemma-server
  template:
    metadata:
      labels:
        app: gemma-server
      annotations:
        gke-gcsfuse/volumes: "true"
    spec:
      containers:
      - name: inference-server
        command: ["python3", "-m", "vllm.entrypoints.openai.api_server"]
        args:
        - --model=$(MODEL)
        - --tensor-parallel-size=1
        - --host=0.0.0.0
        - --port=8000
        env:
        - name: MODEL
          value: /gcs/{V_MODEL_NAME}/{V_MODEL_VERSION}
        - name: VLLM_ATTENTION_BACKEND
          value: FLASHINFER
        image: {V_IMAGE_NAME}
        readinessProbe:
          failureThreshold: 3
          httpGet:
            path: /health
            port: 8000
            scheme: HTTP
          initialDelaySeconds: 240
          periodSeconds: 5
          successThreshold: 1
          timeoutSeconds: 1
        resources:
          requests:
            cpu: "2"
            memory: "25Gi"
            nvidia.com/gpu: "1"
          limits:
            cpu: "2"
            memory: "25Gi"
            nvidia.com/gpu: "1"
        volumeMounts:
        - mountPath: /dev/shm
          name: dshm
        - name: gcs-fuse-csi-ephemeral
          mountPath: /gcs
          readOnly: true
      nodeSelector:
        cloud.google.com/gke-accelerator: nvidia-l4
        cloud.google.com/gke-gpu-driver-version: latest
      serviceAccountName: {V_KSA}
      tolerations:
      - key: "nvidia.com/gpu"
        operator: "Exists"
        effect: "NoSchedule"
      - key: "on-demand"
        value: "true"
        operator: "Equal"
        effect: "NoSchedule" 
      volumes:
      - name: dshm
        emptyDir:
            medium: Memory
      - name: gcs-fuse-csi-ephemeral
        csi:
          driver: gcsfuse.csi.storage.gke.io
          volumeAttributes:
            bucketName: {V_MODEL_BUCKET}
            mountOptions: "implicit-dirs,file-cache:enable-parallel-downloads:true,file-cache:max-parallel-downloads:-1" 
            fileCacheCapacity: "20Gi"
            fileCacheForRangeRead: "true"
            metadataStatCacheCapacity: "-1"
            metadataTypeCacheCapacity: "-1"
            metadataCacheTTLSeconds: "-1"
---
apiVersion: v1
kind: Service
metadata:
  name: llm-service
spec:
  selector:
    app: gemma-server
  type: ClusterIP
  ports:
    - protocol: TCP
      port: 8000
      targetPort: 8000

"""

K8S_YAML_HF=f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: vllm-gemma-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: gemma-server
  template:
    metadata:
      labels:
        app: gemma-server
        ai.gke.io/model: gemma-3-1b-it
        ai.gke.io/inference-server: vllm
        examples.ai.gke.io/source: user-guide
    spec:
      containers:
      - name: inference-server
        image: us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20250312_0916_RC01
        resources:
          requests:
            cpu: "4"
            memory: "30Gi"
            ephemeral-storage: "30Gi"
            nvidia.com/gpu: "1"
          limits:
            cpu: "4"
            memory: "30Gi"
            ephemeral-storage: "30Gi"
            nvidia.com/gpu: "1"
        command: ["python3", "-m", "vllm.entrypoints.openai.api_server"]
        args:
        - --model=$(MODEL_ID)
        - --tensor-parallel-size=1
        - --host=0.0.0.0
        - --port=8000
        env:
        - name: MODEL_ID
          value: google/gemma-3-1b-it
        - name: HUGGING_FACE_HUB_TOKEN
          valueFrom:
            secretKeyRef:
              name: hf-secret
              key: hf_api_token
        volumeMounts:
        - mountPath: /dev/shm
          name: dshm
      volumes:
      - name: dshm
        emptyDir:
            medium: Memory
      nodeSelector:
        cloud.google.com/gke-accelerator: nvidia-l4
        cloud.google.com/gke-gpu-driver-version: latest
---
apiVersion: v1
kind: Service
metadata:
  name: llm-service
spec:
  selector:
    app: gemma-server
  type: ClusterIP
  ports:
    - protocol: TCP
      port: 8000
      targetPort: 8000
"""

with open("vllm-3-1b-it-ft.yaml", "w") as f:
    f.write(K8S_YAML_GCS)

! kubectl apply -f vllm-3-1b-it-ft.yaml

# Wait for container to be created.
import time

print("Waiting for container to be created...\n")
while True:
    shell_output = ! kubectl get pod
    container_status = "\n".join(shell_output)
    if "1/1" in container_status or "Running" in container_status:
        break
    time.sleep(5)

print(container_status)

# Wait for downloading artifacts.
print("\nDownloading artifacts...")
while True:
    shell_output = ! kubectl logs -l app=gemma-server
    logs = "\n".join(shell_output)
    if "Application startup complete" in logs:
        break
    time.sleep(5)

print("Server is up and running.")

### Follow this to server the model

Connect the cluster
Run in Cloud Shell
Open a terminal 

```
kubectl port-forward service/llm-service 8000:8000
```

Open another terminal:
```
curl http://localhost:8080/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "meta-llama/Llama-4-Scout-17B-16E",
    "messages": [{"role": "user", "content": "San Francisco is a"}],
    "max_tokens": 7,
    "temperature": 0
  }'
```


reference: https://cloud.google.com/kubernetes-engine/docs/tutorials/serve-gemma-gpu-vllm#serve-model

## Clean up resources

In [ ]:
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continouous charges that may incur.

! kubectl delete deployments tgi-gemma-deployment
! kubectl delete services llm-service
! kubectl delete secrets hf-secret

DELETE_CLUSTER = False # @param {type: "boolean"}

if DELETE_CLUSTER:
  ! gcloud container clusters delete {CLUSTER_NAME} \
    --region={REGION} \
    --quiet